In [2]:
using Revise, REopt, Xpress, Pkg, JSON, JuMP, CSV, DataFrames, Formatting, HTTP, Base64, Dates, TimeSeries, Statistics, TimeZones, Plots
ENV["NREL_DEVELOPER_API_KEY"]=""
Pkg.status()

Status `~/Documents/GitHub/tldrd_mpc/Project.toml`
⌅ [c9ce4bd3] ArchGDAL v0.9.4
  [336ed68f] CSV v0.10.15
  [9961bab8] Cbc v1.2.0
  [a93c6f00] DataFrames v1.7.0
  [59287772] Formatting v0.4.3
  [cd3eb016] HTTP v1.10.15
  [682c06a0] JSON v0.21.4
  [4076af6c] JuMP v1.23.5
⌃ [91a5bcdd] Plots v1.40.7
  [d36ad4e8] REopt v0.47.2 `~/.julia/dev/REopt`
  [295af30f] Revise v3.7.1
  [9e3dc215] TimeSeries v0.24.2
  [f269a46b] TimeZones v1.20.0
  [9e70acf3] Xpress v0.17.1
  [ade2ca70] Dates
  [10745b16] Statistics v1.10.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


### Test MPC functionality

In [20]:
# Paths
inputs_path = "inputs"

# Filenames
load_file = "example_electric_load.csv"
energy_cost_file = "example_energy_cost_series.csv"
demand_cost_file = "example_tou_demand.csv"

pv_amy_prod_factors_file = "AMY_PV_production_factor.csv" 
wind_amy_prod_factors_file = "AMY_wind_production_factor.csv"
TMY_prod_factors_file = "TMY_production_factors.csv" 

emissions_file = "example_hourly_moer.csv"

# Load relevant csvs 
full_load = CSV.read(inputs_path * "/" * load_file, DataFrame)
energy_cost = CSV.read(inputs_path * "/" * energy_cost_file, DataFrame)
demand_cost = CSV.read(inputs_path * "/" * demand_cost_file, DataFrame)

pv_amy_prod_factor = CSV.read(inputs_path * "/" * pv_amy_prod_factors_file, DataFrame)
wind_amy_prod_factor = CSV.read(inputs_path * "/" * wind_amy_prod_factors_file, DataFrame)
TMY_prod_factors = CSV.read(inputs_path * "/" * TMY_prod_factors_file, DataFrame)

hourly_emissions = CSV.read(inputs_path * "/" * emissions_file, DataFrame)


## Scenario Inputs
looping_method = "perfect" # perfect, imperfect_1

save_results = true
outputs_path = "outputs"
filename = "test_example" 
wind_in_scenario = true

allow_export = false

reopt_resource_type = "8760" # TMY, actual_year, 8760 (strings)
new_year_starting_ts = 25

include_climate_in_objective = false

## Hydrogen and Thermal Loads
hydrogen_load_kg_per_hour = repeat([250], 8760) 
heat_loads_mmbtu_per_hour = repeat([17], 8760) 

## Technology Inputs
pv_kw = 19000 
bess_kw = 20000
bess_kwh = 800000

soc_init_fraction = 0.2
soc_min_fraction = 0.2
internal_efficiency_fraction = 0.975
inverter_efficiency_fraction = 0.96
rectifier_efficiency_fraction = 0.96

charge_efficiency = rectifier_efficiency_fraction * internal_efficiency_fraction^0.5
discharge_efficiency = inverter_efficiency_fraction * internal_efficiency_fraction^0.5

electric_heater_mmbtu_per_hour = 20
htts_charge_kw = 100000.0
htts_discharge_kw = 10000.0 
htts_kwh = 100000.0

electrolyzer_kw = 20000
require_compression = false
fuel_cell_kw = 10000
hydrogen_storage_kg = 10000    

#########################################################

energy_rate = energy_cost[!,"usd_per_kwh"]
demand_cost = demand_cost[!,"usd_per_kw"]
month_transition_timesteps = [744, 1416, 2160, 2880, 3624, 4344, 5088, 5832, 6552, 7296, 8016, 8760]

load = full_load[!,"loads_kW"]
pv_prod_factor_forecast = pv_amy_prod_factor[!,"pv_2022"]
wind_prod_factor_forecast = wind_amy_prod_factor[!,"wind_2013"]

pv_prod_factor_TMY = TMY_prod_factors[!,"pv_prod_factor"]
wind_prod_factor_TMY = TMY_prod_factors[!,"wind_prod_factor"]

emissions_lbs_per_kwh = hourly_emissions[!,"MOER [lbsCO2/kWh]"]

tou_demand_rates = [0.0, 0.0, 0.0, 0.0]
tou_demand_time_steps = [[], []]
tou_previous_peak_demands = [0, 0]; 

In [22]:
post = Dict([
    ("Settings", Dict([
        ("include_climate_in_objective", include_climate_in_objective),
        ])
    ),
    ("Site", Dict([
        ("include_exported_elec_emissions_in_total", true),
        ])
    ),
    ("ElectricLoad", Dict([
        ("loads_kw", [])
        ])),
    ("ElectricTariff",Dict([ 
       ("energy_rates", []),
       ("tou_demand_rates", tou_demand_rates),
       ("tou_demand_time_steps", tou_demand_time_steps),
       ("tou_previous_peak_demands", tou_previous_peak_demands) 
        ])), 
    ("ElectricUtility", Dict([
        ("emissions_factor_series_lb_CO2_per_kwh", [0.0]),
        ("emissions_factor_series_lb_NOx_per_kwh", [0.0]),
        ("emissions_factor_series_lb_SO2_per_kwh", [0.0]),
        ("emissions_factor_series_lb_PM25_per_kwh", [0.0])
        ])
    ),
#     ("Financial", Dict([
#         ("CO2_cost_per_tonne", 100.0)
#         ]
#         )),
    ("PV", Dict([
        ("size_kw", pv_kw),
        ("production_factor_series", [])
        ]
        )), 
    ("Wind", Dict([
        ("size_kw", wind_kw),
        ("production_factor_series", [])
        ]
        )), 
    ("ElectricStorage",Dict([
        ("size_kw", bess_kw), 
        ("size_kwh", bess_kwh), 
        ("charge_efficiency", charge_efficiency),
        ("discharge_efficiency", discharge_efficiency),
        ("soc_init_fraction", 0.2),
        ("soc_min_fraction", soc_min_fraction)
        ])), 
    ("ProcessHeatLoad", Dict([
        ("heat_loads_mmbtu_per_hour", heat_loads_mmbtu_per_hour),
        ])
    ),
    ("ElectricHeater", Dict([
        ("size_mmbtu_per_hour", electric_heater_mmbtu_per_hour)
        ])
    ),
    ("HighTempThermalStorage", Dict([
        ("charge_kw", htts_charge_kw),
        ("discharge_kw", htts_discharge_kw),
        ("size_kwh", htts_kwh)
        ])
    ),
    ("Electrolyzer", Dict([
        ("size_kw", electrolyzer_kw),
        ("require_compression", require_compression)
        ])
    ),
    ("FuelCell", Dict([
        ("size_kw", fuel_cell_kw),
        ])
    ), 
    ("HydrogenStorage",Dict([
        ("size_kg", hydrogen_storage_kg),
        ])
    ),
    ("HydrogenLoad",Dict([
        ("loads_kg", hydrogen_load_kg_per_hour),
        ])
    )
    ]);

In [23]:
function get_tou_demand_time_steps(cost_array, ts_array)
    tou_demand_ts = []
    for cost in cost_array
        push!(tou_demand_ts, findall(x->x==cost, ts_array))
    end
    return tou_demand_ts
end

get_tou_demand_time_steps (generic function with 1 method)

In [24]:
function dispatch_method_1(result, r1_ts, r2_ts, bess_kwh, previous_soc, charge_efficiency, discharge_efficiency, hours_per_time_step)

    # Process control execution
    load_ts = result["ElectricLoad"]["load_series_kw"][1]
    bess_dispatch_ts = result["ElectricStorage"]["storage_to_load_series_kw"][1] - 
                            result["PV"]["electric_to_storage_series_kw"][1] - 
                            result["Wind"]["electric_to_storage_series_kw"][1] - 
                            result["ElectricUtility"]["electric_to_storage_series_kw"][1]

    # RE resources are greater than the load (resource 1 is prioritized)
    if (r1_ts + r2_ts) >= load_ts
        util_to_load = 0
        bess_to_load = 0

        # Resource 1 generates enough to serve load by itself
        if r1_ts >= load_ts
            r1_to_load = load_ts
            r1_excess = r1_ts - r1_to_load
            r2_to_load = 0
            r2_excess = r2_ts

        # Both resources are need to serve load
        else 
            r1_to_load = r1_ts
            r1_excess = 0
            r2_to_load = load_ts - r1_to_load
            r2_excess = r2_ts - r2_to_load
        end

        # Control signal - Battery discharge
        if bess_dispatch_ts >= 0
            r1_to_bess = 0
            r2_to_bess = 0
            util_to_bess = 0
            bess_soc = previous_soc

         # Control signal - Battery charge
        else
            bess_soc = min(1.0, previous_soc + (abs(bess_dispatch_ts) * charge_efficiency * hours_per_time_step) / bess_kwh)

            # Resource 1 generates enough to charge battery after serving load
            if r1_excess > abs(bess_dispatch_ts)
                r1_to_bess = abs(bess_dispatch_ts)
                r1_excess = r1_excess - abs(bess_dispatch_ts)
                r2_to_bess = 0
                util_to_bess = 0 

            # Resource 1 and 2 together generate enough to charge battery after serving load
            elseif (r1_excess + r2_excess) > abs(bess_dispatch_ts)
                r1_to_bess = r1_excess
                r1_excess = 0
                r2_to_bess = abs(bess_dispatch_ts) - r1_to_bess
                r2_excess = r2_excess - r2_to_bess
                util_to_bess = 0

            # Excess RE is insufficient to charge battery    
            else 
                r1_to_bess = r1_excess
                r1_excess = 0
                r2_to_bess = r2_excess
                r2_excess = 0
                util_to_bess = abs(bess_dispatch_ts) - r1_to_bess - r2_to_bess
            end
        end

    # RE resouces are not enough to meet load
    else 
        r1_to_load = r1_ts
        r1_excess = 0
        r1_to_bess = 0
        r2_to_load = r2_ts
        r2_excess = 0            
        r2_to_bess = 0
        unmet_load = load_ts - r1_ts - r2_ts

        # Control signal - Battery discharge
        if bess_dispatch_ts >= 0
            util_to_bess = 0

            # Optimal battery discharge signal can meet the load
            if bess_dispatch_ts >= unmet_load
                bess_to_load = unmet_load
                util_to_load = 0
                bess_soc = max(soc_min_fraction, previous_soc - ((unmet_load/discharge_efficiency) * hours_per_time_step) / bess_kwh)

            # Optimal battery dispatch signal cannot meet the load
            else 
                bess_to_load = bess_dispatch_ts
                util_to_load = unmet_load - bess_dispatch_ts
                bess_soc = max(soc_min_fraction, previous_soc - ((bess_dispatch_ts/discharge_efficiency) * hours_per_time_step) / bess_kwh)
            end

        # Control signal - Battery charge  
        else 
            util_to_load = unmet_load
            util_to_bess = abs(bess_dispatch_ts)
            bess_to_load = 0
            bess_soc = min(1.0, previous_soc + (abs(bess_dispatch_ts) * charge_efficiency * hours_per_time_step) / bess_kwh)
        end
    end

    return r1_to_load, r1_to_bess, r1_excess, r2_to_load, r2_to_bess, r2_excess, bess_to_load, bess_soc, util_to_load, util_to_bess
end

dispatch_method_1 (generic function with 1 method)

In [25]:
# MPC Algorithm

hours_per_time_step = 1
horizon = 168
stopping_ts = 2 # 8760 for full year run
starting_ts = 1
length_of_data = 8760

# Running sum variables
energy_cost_kwh = 0
bess_soc_init_fraction = 0.2
htts_init_fraction = 0.5
hess_init_fraction = 0.5

tou_previous_peak_demands = zeros(length(tou_demand_rates))
monthly_demand_cost = []

tou_peaks_by_month = []

# Full results from each MPC loop
results = []

# Executed dispatch results 
if wind_in_scenario
    dispatch_results = Dict([
        ("PV", Dict([
            ("electric_to_load_series_kw", []),
            ("electric_to_storage_series_kw", []),
            ("electric_to_grid_series_kw", []),
            ("electric_curtailed_series_kw", [])
            ])
        ),
        ("Wind", Dict([
            ("electric_to_load_series_kw", []),
            ("electric_to_storage_series_kw", []),
            ("electric_to_grid_series_kw", []),
            ("electric_curtailed_series_kw", [])
            ])
        ),
        ("ElectricStorage", Dict([
            ("storage_to_load_series_kw", []),
            ("soc_series_fraction", [])
            ])
        ),        
        ("ElectricUtility", Dict([
            ("electric_to_load_series_kw", []),
            ("electric_to_storage_series_kw", []),
            ("emissions_series_tonnes_CO2", [])           
            ])
        ),
        ("ElectricLoad", Dict([
            ("load_series_kw", []),
            ])
        )
    ])
else
    dispatch_results = Dict([
        ("PV", Dict([
            ("electric_to_load_series_kw", []),
            ("electric_to_storage_series_kw", []),
            ("electric_to_grid_series_kw", []),
            ("electric_curtailed_series_kw", [])
            ])
        ),
        ("ElectricStorage", Dict([
            ("storage_to_load_series_kw", []),
            ("soc_series_fraction", [])
            ])
        ),        
        ("ElectricUtility", Dict([
            ("electric_to_load_series_kw", []),
            ("electric_to_storage_series_kw", []),
            ("emissions_series_tonnes_CO2", [])           
            ])
        ),
        ("ElectricLoad", Dict([
            ("load_series_kw", []),
            ])
        ),
        ("HeatingLoad", Dict([
            ("process_heat_thermal_load_series_mmbtu_per_hour", []),
            ])
        ),
        ("ElectricHeater", Dict([
            ("electric_consumption_series_kw", []),
            ("thermal_production_series_mmbtu_per_hour", []),
            ("thermal_to_storage_series_mmbtu_per_hour", []),
            ("thermal_to_load_series_mmbtu_per_hour", []),
            ("thermal_to_process_heat_load_series_mmbtu_per_hour", []),
            ])
        ),
        ("HighTempThermalStorage", Dict([
            ("soc_series_fraction", []),
            ("storage_to_load_series_mmbtu_per_hour", []),
            ("storage_to_process_heat_load_series_mmbtu_per_hour", [])
            ])
        ),
        ("HydrogenLoad", Dict([
            ("load_series_kg", []),
            ])
        ),
        ("Electrolyzer", Dict([
            ("electricity_consumed_series_kw", []),
            ("hydrogen_produced_series_kg", [])
            ])
        ),
        ("HydrogenStorage", Dict([
            ("soc_series_fraction", []),
            ("discharge_from_storage_series_kg", [])
            ])
        ),
        ("FuelCell", Dict([
            ("hydrogen_consumed_series_kg", []),
            ("electric_to_load_series_kw", []),
            ("electric_to_storage_series_kw", []),
            ])
        )
    ])
end

cost_series = []
for idx in range(starting_ts, stop=stopping_ts)
    
    end_ts = idx+horizon-1
    if end_ts > length_of_data
        end_ts = length_of_data
    end
    
    # Update forecast for each run
    if reopt_resource_type == "8760"        
        
        post["PV"]["production_factor_series"] = [pv_prod_factor_forecast[idx:end_ts]; 
                                                  pv_prod_factor_TMY[end_ts+1:length(pv_prod_factor_TMY)]; 
                                                  pv_prod_factor_TMY[1:idx-1]]
        if wind_in_scenario
            post["Wind"]["production_factor_series"] = [wind_prod_factor_forecast[idx:end_ts];
                                                        wind_prod_factor_TMY[end_ts+1:length(wind_prod_factor_TMY)];
                                                        wind_prod_factor_TMY[1:idx-1]]
        end
        
        splice_end_ts = new_year_starting_ts + idx - 2
        new_end_ts = 0
        if splice_end_ts > length_of_data
            new_end_ts = new_year_starting_ts + splice_end_ts - length_of_data - 1
            splice_end_ts = length_of_data
        end

        post["ElectricTariff"]["energy_rates"] = [energy_rate[idx:length(energy_rate)];
                                                  energy_rate[new_year_starting_ts:splice_end_ts];
                                                  energy_rate[new_year_starting_ts:new_end_ts]]
        post["ElectricLoad"]["loads_kw"] = [load[idx:length(load)];
                                            load[new_year_starting_ts:splice_end_ts];
                                            load[new_year_starting_ts:new_end_ts]]
        post["ElectricUtility"]["emissions_factor_series_lb_CO2_per_kwh"] = [emissions_lbs_per_kwh[idx:length(emissions_lbs_per_kwh)];
                                           emissions_lbs_per_kwh[new_year_starting_ts:splice_end_ts];
                                           emissions_lbs_per_kwh[new_year_starting_ts:new_end_ts]]
    else
        post["PV"]["production_factor_series"] = pv_prod_factor[idx:end_ts]
        post["Wind"]["production_factor_series"] = wind_prod_factor[idx:end_ts]
        
        post["ElectricTariff"]["energy_rates"] = energy_rate[idx:end_ts]
        post["ElectricLoad"]["loads_kw"] = load[idx:end_ts]
        post["ElectricUtility"]["emissions_factor_series_lb_CO2_per_kwh"] = emissions_lbs_per_kwh[idx:end_ts]
    end
    
    post["ElectricStorage"]["soc_init_fraction"] = bess_soc_init_fraction
    post["HydrogenStorage"]["soc_init_fraction"] = hess_soc_init_fraction
    post["HighTempThermalStorage"]["soc_init_fraction"] = htts_soc_init_fraction
        
    if rate_scenario == "tou"
        tou_demand_time_steps = get_tou_demand_time_steps(tou_demand_rates, demand_cost[idx:end_ts])
        
#         post["ElectricTariff"]["tou_demand_rates"] = tou_demand_rates # Set earlier, doesn't change by loop
        post["ElectricTariff"]["tou_demand_time_steps"] = tou_demand_time_steps
        post["ElectricTariff"]["tou_previous_peak_demands"] = tou_previous_peak_demands
    end

    # Run optimization
    model = Model(Xpress.Optimizer)
    result = run_mpc(model, post)
    
    # Save full set of results per MPC loop 
    push!(results, result) 
    
    # Get results needed for the next iteration of the MPC algorithm
    if looping_method == "perfect"
        
        r1_to_load = result["PV"]["electric_to_load_series_kw"][1]
        r1_to_bess = result["PV"]["electric_to_storage_series_kw"][1]
        # Not sure if REopt MPC ever sends anything to grid
        # Currently assumes all excess is exported or curtailed (export benefits not calculated though)
        r1_excess = result["PV"]["electric_to_grid_series_kw"][1] + result["PV"]["electric_curtailed_series_kw"][1]
        
        if wind_in_scenario
            r2_to_load = result["Wind"]["electric_to_load_series_kw"][1]
            r2_to_bess = result["Wind"]["electric_to_storage_series_kw"][1]
            r2_excess = result["Wind"]["electric_to_grid_series_kw"][1] + result["Wind"]["electric_curtailed_series_kw"][1]
        end
        
        bess_to_load = result["ElectricStorage"]["storage_to_load_series_kw"][1]
        bess_soc = result["ElectricStorage"]["soc_series_fraction"][1]
        util_to_load = result["ElectricUtility"]["electric_to_load_series_kw"][1]
        util_to_bess = result["ElectricUtility"]["electric_to_storage_series_kw"][1]

        grid_power = max(result["ElectricUtility"]["electric_to_load_series_kw"][1] + 
                         result["ElectricUtility"]["electric_to_storage_series_kw"][1], 0)      
        
    elseif looping_method == "imperfect_1"
        
        r1_to_load, r1_to_bess, r1_excess, r2_to_load, r2_to_bess, r2_excess, bess_to_load, bess_soc, util_to_load, util_to_bess = dispatch_method_1(result, realized_pv_prod_factor[idx]*pv_kw, realized_wind_prod_factor[idx]*wind_kw, bess_kwh, soc_init_fraction, charge_efficiency, discharge_efficiency, hours_per_time_step)
        grid_power = max(util_to_load + util_to_bess, 0)
         
    else
        print("Undefined Looping Method")
        break
    end
    
    # Process time series results for the timestep executed 
    push!(dispatch_results["PV"]["electric_to_load_series_kw"], r1_to_load)
    push!(dispatch_results["PV"]["electric_to_storage_series_kw"], r1_to_bess)
    if allow_export
        push!(dispatch_results["PV"]["electric_to_grid_series_kw"], r1_excess)
        push!(dispatch_results["PV"]["electric_curtailed_series_kw"], 0)
    else
        push!(dispatch_results["PV"]["electric_to_grid_series_kw"], 0)
        push!(dispatch_results["PV"]["electric_curtailed_series_kw"], r1_excess)
    end
    
    if wind_in_scenario
        push!(dispatch_results["Wind"]["electric_to_load_series_kw"], r2_to_load)
        push!(dispatch_results["Wind"]["electric_to_storage_series_kw"], r2_to_bess)
        if allow_export
            push!(dispatch_results["Wind"]["electric_to_grid_series_kw"], r2_excess)
            push!(dispatch_results["Wind"]["electric_curtailed_series_kw"], 0)
        else
            push!(dispatch_results["Wind"]["electric_to_grid_series_kw"], 0)
            push!(dispatch_results["Wind"]["electric_curtailed_series_kw"], r2_excess)            
        end
    end
    
    push!(dispatch_results["ElectricStorage"]["storage_to_load_series_kw"], bess_to_load)
    push!(dispatch_results["ElectricStorage"]["soc_series_fraction"], round(bess_soc, digits=6))

    push!(dispatch_results["ElectricUtility"]["electric_to_load_series_kw"], util_to_load)
    push!(dispatch_results["ElectricUtility"]["electric_to_storage_series_kw"], util_to_bess) 
    push!(dispatch_results["ElectricUtility"]["emissions_series_tonnes_CO2"], emissions_lbs_per_kwh[idx] * grid_power * hours_per_time_step) 
    
    push!(dispatch_results["ElectricLoad"]["load_series_kw"], result["ElectricLoad"]["load_series_kw"][1])

    push!(dispatch_results["HeatingLoad"]["process_heat_thermal_load_series_mmbtu_per_hour"], result["HeatingLoad"]["process_heat_thermal_load_series_mmbtu_per_hour"][1])
    
    push!(dispatch_results["ElectricHeater"]["electric_consumption_series_kw"], result["ElectricHeater"]["electric_consumption_series_kw"][1])
    push!(dispatch_results["ElectricHeater"]["thermal_production_series_mmbtu_per_hour"], result["ElectricHeater"]["thermal_production_series_mmbtu_per_hour"][1])
    push!(dispatch_results["ElectricHeater"]["thermal_to_storage_series_mmbtu_per_hour"], result["ElectricHeater"]["thermal_to_storage_series_mmbtu_per_hour"][1])
    push!(dispatch_results["ElectricHeater"]["thermal_to_load_series_mmbtu_per_hour"], result["ElectricHeater"]["thermal_to_load_series_mmbtu_per_hour"][1])
    push!(dispatch_results["ElectricHeater"]["thermal_to_process_heat_load_series_mmbtu_per_hour"], result["ElectricHeater"]["thermal_to_process_heat_load_series_mmbtu_per_hour"][1])
    
    push!(dispatch_results["HighTempThermalStorage"]["soc_series_fraction"], result["HighTempThermalStorage"]["soc_series_fraction"][1])
    push!(dispatch_results["HighTempThermalStorage"]["storage_to_load_series_mmbtu_per_hour"], result["HighTempThermalStorage"]["storage_to_load_series_mmbtu_per_hour"][1])
    push!(dispatch_results["HighTempThermalStorage"]["storage_to_process_heat_load_series_mmbtu_per_hour"], result["HighTempThermalStorage"]["storage_to_process_heat_load_series_mmbtu_per_hour"][1])
    
    push!(dispatch_results["HydrogenLoad"]["load_series_kg"], result["HydrogenLoad"]["load_series_kg"][1])
    
    push!(dispatch_results["Electrolyzer"]["electricity_consumed_series_kw"], result["Electrolyzer"]["electricity_consumed_series_kw"][1])
    push!(dispatch_results["Electrolyzer"]["hydrogen_produced_series_kg"], result["Electrolyzer"]["hydrogen_produced_series_kg"][1])

    push!(dispatch_results["HydrogenStorage"]["soc_series_fraction"], result["HydrogenStorage"]["soc_series_fraction"][1])
    push!(dispatch_results["HydrogenStorage"]["discharge_from_storage_series_kg"], result["HydrogenStorage"]["discharge_from_storage_series_kg"][1])
    
    push!(dispatch_results["FuelCell"]["hydrogen_consumed_series_kg"], result["FuelCell"]["hydrogen_consumed_series_kg"][1])
    push!(dispatch_results["FuelCell"]["electric_to_load_series_kw"], result["FuelCell"]["electric_to_load_series_kw"][1])
    push!(dispatch_results["FuelCell"]["electric_to_storage_series_kw"], result["FuelCell"]["electric_to_storage_series_kw"][1])
    
    
    # Calculate running sums of relevant values
    energy_cost_kwh += grid_power * post["ElectricTariff"]["energy_rates"][1]
    push!(cost_series, grid_power * post["ElectricTariff"]["energy_rates"][1])
    
    bess_soc_init_fraction = bess_soc
    hess_soc_init_fraction = result["HydrogenStorage"]["soc_series_fraction"][1]
    htts_soc_init_fraction = result["HighTempThermalStorage"]["soc_series_fraction"][1]
    
    if rate_scenario == "tou"
        # Find which time of use period the current timestep is in
        i = findfirst(==(demand_cost[idx]), tou_demand_rates)
        tou_previous_peak_demands[i] = max(grid_power, tou_previous_peak_demands[i])
        
        # Calculate the total TOU demand charge at the end of each month
        # Currently, MPC returns demand charge = 0 if run for less than 1 month
        if idx in month_transition_timesteps
            append!(monthly_demand_cost, tou_previous_peak_demands .* tou_demand_rates)
            push!(tou_peaks_by_month, tou_previous_peak_demands)
            tou_previous_peak_demands = zeros(length(tou_demand_rates))
        end
    end
    
end

print("\n\nFinished loop\n")

[ Info: Model built. Optimizing...


FICO Xpress v8.12.3, Hyper, solve started 17:57:24, Sep 1, 2025
Heap usage: 115MB (peak 115MB, 893KB system)
Minimizing LP  with these control settings:
OUTPUTLOG = 1
MPSNAMELENGTH = 64
CALLBACKFROMMASTERTHREAD = 1
Original problem has:
    402964 rows       359172 cols      1147458 elements
Presolved problem has:
    109438 rows       122524 cols       371793 elements
Presolve finished in 2 seconds
Heap usage: 130MB (peak 344MB, 895KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.48e-04,  1.11e+00] / [ 2.50e-01,  1.00e+00]
  RHS and bounds [min,max] : [ 2.50e+02,  8.00e+05] / [ 3.52e-01,  1.60e+06]
  Objective      [min,max] : [ 1.14e-04,  1.52e-01] / [ 7.13e-06,  1.52e-01]
Autoscaling applied Curtis-Reid scaling

Crash basis containing 1 structural columns created
Starting parallel dual simplex, using up to 8 threads
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0      -22187400.61    

┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 42.828 seconds.
[ Info: Results processing took 12.409 seconds.
[ Info: Results processing took 12.41 seconds.
[ Info: Model built. Optimizing...


FICO Xpress v8.12.3, Hyper, solve started 17:58:29, Sep 1, 2025
Heap usage: 115MB (peak 115MB, 723KB system)
Minimizing LP  with these control settings:
OUTPUTLOG = 1
MPSNAMELENGTH = 64
CALLBACKFROMMASTERTHREAD = 1
Original problem has:
    402964 rows       359172 cols      1147458 elements
Presolved problem has:
    109438 rows       122524 cols       371793 elements
Presolve finished in 2 seconds
Heap usage: 130MB (peak 344MB, 726KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.48e-04,  1.11e+00] / [ 2.50e-01,  1.00e+00]
  RHS and bounds [min,max] : [ 2.50e+02,  8.00e+05] / [ 3.52e-01,  1.60e+06]
  Objective      [min,max] : [ 1.14e-04,  1.52e-01] / [ 7.13e-06,  1.52e-01]
Autoscaling applied Curtis-Reid scaling

Crash basis containing 1 structural columns created
Starting parallel dual simplex, using up to 8 threads
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0      -22187387.23    

┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1




Finished loop


[ Info: Solving took 42.983 seconds.
[ Info: Results processing took 8.695 seconds.
[ Info: Results processing took 8.697 seconds.


In [26]:
if save_results
    open(outputs_path * "/" * filename * ".json","w") do f
      JSON.print(f, dispatch_results)
    end
end

In [67]:
df = DataFrame()

# df[!, "pv_prod_factor"] = pv_prod_factor
df[!, "pv_to_load_series_kw"] = dispatch_results["PV"]["electric_to_load_series_kw"]
df[!, "pv_to_battery_series_kw"] = dispatch_results["PV"]["electric_to_storage_series_kw"]
df[!, "pv_to_grid_series_kw"] = dispatch_results["PV"]["electric_to_grid_series_kw"]
df[!, "pv_curtailed_production_series_kw"] = dispatch_results["PV"]["electric_curtailed_series_kw"]

# df[!, "wind_prod_factor"] = wind_prod_factor
df[!, "wind_to_load_series_kw"] = dispatch_results["Wind"]["electric_to_load_series_kw"]
df[!, "wind_to_battery_series_kw"] = dispatch_results["Wind"]["electric_to_storage_series_kw"]
df[!, "wind_to_grid_series_kw"] = dispatch_results["Wind"]["electric_to_grid_series_kw"]
df[!, "wind_curtailed_production_series_kw"] = dispatch_results["Wind"]["electric_curtailed_series_kw"]

df[!, "storage_to_load_series_kw"] = dispatch_results["ElectricStorage"]["storage_to_load_series_kw"]
df[!, "soc_series_fraction"] = dispatch_results["ElectricStorage"]["soc_series_fraction"]

df[!, "grid_to_load_series_kw"] = dispatch_results["ElectricUtility"]["electric_to_load_series_kw"]
df[!, "grid_to_battery_series_kw"] = dispatch_results["ElectricUtility"]["electric_to_storage_series_kw"]
df[!, "grid_emissions_tonnes"] = dispatch_results["ElectricUtility"]["emissions_series_tonnes_CO2"]

if stopping_ts == 8760
    df[!, "emissions_factor_lbs_per_kwh"] = emissions_lbs_per_kwh
end

df[!, "load_series_kw"] = dispatch_results["ElectricLoad"]["load_series_kw"]
df[!, "cost_per_timestep"] = cost_series

if save_results
    CSV.write(outputs_path * "/" * filename * ".csv", df)
end

"outputs/feedback_loop/perfect_feedback_tou.csv"